# Исследование надежности заемщиков

***Цель исследования*** - проверить как влияет наличие детей, семейное положение, доход и цель получения кредита заемщика на факт погашения кредита в срок


***Описание исследования***

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***Ход исследования***

Мы проверим данные на ошибки. Затем, на этапе предобработки поищем возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

Обзор данных.
Предобработка данных.
Проверка гипотез.

В ходе проверки гипотез ответим на вопросы:
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Обзор данных

Импортируем данную библиотеку pandas для работы с таблицей:

In [1]:
# импорт библиотеки pandas
import pandas as pd

Прочитаем файл data.csv из папки /datasets и сохраним его в переменной data

In [2]:
# чтение файла с данными и сохранение в переменной data
data = pd.read_csv('/datasets/data.csv')

Ознакомимся с данными. Выведем на экран первые 10 строк таблицы:

In [3]:
# получение первых 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [4]:
# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Типы данных - object, int и float. 

Согласно документации к данным:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

В названиях колонок нарушений не вижу. 

Исходя из выведенных данных заметны:
- аномалии в количестве дней трудового стажа (отрицательные данные)
- разный регистр букв в столбце образования
- цели получения кредита написаны по-разному хотя по сути одинаковые. Например, "сыграть свадьбу" и "на проведение свадьбы"

Количество значений в столбцах days_employed (общий трудовой стаж) и total_income (ежемесячный доход) отличаются от остальных. Можно предположить, что данные кредиты выданы не работающим гражданам. Это необходимо проверить дополнительно поняв какой income_type у таких граждан. 

***Вывод***

В таблице представлены данные о заемщиках банка - причины взятия кредита, их статус, наличие детей, данные об образовании, поле и т.д. А также есть информация по наличию задолженности по возврату кредита. 
Предварительно этих данных должно быть достаточно для ответа на вопросы выше - о связи семейного статуса и наличия детей с задолженностью. 
В данных есть аномалии, также их требуется привести к хорошему стилю для проверки гипотез. 
Возможно наличие дубликатов, есть пропуски. 

### Шаг 2.1 Заполнение пропусков

Начинаем предобработку данных с пропущенных значений. 

Пропущенные значения находятся в столбцах days_employed и total_income. 
Есть несколько версий причин пропусков значений:
- человеческий фактор (случайные пропуски значений при заполнении)
- для заполнения этих данных требовалась справка с места работы, например, и ее не было на руках в момент подачи заявки и заполнения данных
- данные граждане принадлежат к категориям не работающих граждан
- данные граждане не трудоустроены официально, поэтому данных об их стаже и ежемесячном доходе нет в базе

Посчитаем общее количество пропущенных значений в столбцах days_employed и total_income:

In [5]:
# подсчет пропущенных значений в столбце days_employed
len(data[data['days_employed'].isna()])

2174

In [6]:
# подсчет пропущенных значений в столбце total_income
len(data[data['total_income'].isna()])

2174

Количество пропущенных значений в столбцах одинаковое. Посчитаем долю пропущенных значений от общего числа значений в столбце.
Поскольку ранее при вызове функции info() было видно, что общее количество значений во всех столбцах одинаковое сделаем это на примере одного из столбцов с пропусками.

In [7]:
# расчитываем долю пропущенных значений от общего количества значений в столбце, на примере столбца total_income
share_total_income = len(data[data['total_income'].isna()])/len(data['total_income'])
print(f'{share_total_income:.0%}')

10%


Количество пропусков в обоих столбцах равно 10% от общего количества значений.

Попробуем сгруппировать данные по столбцу income_type (тип занятости), чтобы проверить гипотезу о том, что кредиты были выданы неработающим гражданам, в связи с чем есть отсутствующие значения.

In [8]:
# группируем данные по столбцу income_type
income_type_group = data.groupby('income_type')
display(income_type_group.count())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
в декрете,1,1,1,1,1,1,1,1,1,1,1
госслужащий,1459,1312,1459,1459,1459,1459,1459,1459,1459,1312,1459
компаньон,5085,4577,5085,5085,5085,5085,5085,5085,5085,4577,5085
пенсионер,3856,3443,3856,3856,3856,3856,3856,3856,3856,3443,3856
предприниматель,2,1,2,2,2,2,2,2,2,1,2
сотрудник,11119,10014,11119,11119,11119,11119,11119,11119,11119,10014,11119
студент,1,1,1,1,1,1,1,1,1,1,1


Гипотеза не подтвердилась - большинство данных пропущены у таких групп как госслужащий, компаньон, сотрудник. 
В тоже время у безработных граждан, граждан в декрете, студентов данные присутствуют. 

Соответственно большая вероятность, что пропуски значений связаны с человеческим факторов или отсутствием официальной занятости у указанных заемщиков. Для продолжения исследования необходимо устранить пропуски. 

Для устранения пропусков можно использовать средние и медианные значения. 
В случае количественных данных медианные значения предпочтительнее, так как среднее значение может быть не показательным из-за сильно разброса данных. 

Заменим пропущенные значения в столбцах медианным значением и проверим результат замены:

In [9]:
# ищем медианное значение столбца days_employed
days_employed_madian = data['days_employed'].median()
# заменяем пропущенные значения в столбце медианным
data['days_employed'] = data['days_employed'].fillna(days_employed_madian)
# проверяем результат замены
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
# ищем медианное значение столбца total_income
total_income_madian = data['total_income'].median()
# заменяем пропущенные значения в столбце медианным
data['total_income'] = data['total_income'].fillna(total_income_madian)
# проверяем результат замены
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


***Вывод***

В столбцах days_employed и total_income были пропущены примерно 10% от всех значений. 
Мы предположили, что указанные значения были пропущены, так как эти заемщики относились к категориям неработающих граждан. Данная гипотеза не подтвердилась. 

Скорее всего значения пропущены случайно или у данных граждан нет официального места работы, возможно они не предоставили подтверждающих документов. 

Поскольку у нас нет возможности уточнить эти данные мы можем заменить пропущенные значения либо средними, либо медианными. 
Медианные значения подходят для количественных данных лучше, так как дают более точную картину данных в условиях большого их разброса. 

Мы произвели замену пустых значений, и готовы к дальнейшей обработке. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут присутствовать аномалии, которые необходимо поправить для корректной проверки гипотез. В данной таблице аномальные данные представлены, например, в столбце days_employed в виде отрицательных значений дней стажа. 

В целом отрицательные значений скорее ошибка в виде человеческого фактора. Действительно, при расчете количества дней трудового стажа могут появляться отрицательные значения при вычитании дат поступления на работу и увольнения. Вероятно, при переводе в дни не были убраны данные отрицательные значения. 

Приведем все значения к положительным:

In [11]:
# приводим все значения столбца days_employed к абсолютным и 
data['days_employed'] = data['days_employed'].abs()

Проверим наличие отрицательных значений в столбце data_employed после обработки данных:

In [12]:
# выводим первые 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Отлично. Значения в столбце стали абсолютными. Проверим какие значения выпадают из общих данных. Проверим, какие есть минимальные значения в столбцах таблицы:

In [13]:
# ищем минимальные значения в столбцах таблицы data
data.min()

children                               -1
days_employed                     24.1416
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Исходя из данных видно, что иногда встречается отрицательное количество детей, что некорректно. Вероятно, это является также ошибкой при заполнении данных. Приведем все значения столбца children к абсолютным:

In [14]:
# приводим значения столбца children к абсолютным
data['children'] = data['children'].abs()

Проверим значения столбца children после внесения правок:

In [15]:
# выводим все уникальные значения столбца children
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Отрицательные значения количества детей поправили. Проверим еще раз минимальные значения в таблице:

In [16]:
# повторная проверка минимальных значений
data.min()

children                                0
days_employed                     24.1416
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Минимальное значение похоже теперь на реальные данные. Обработка прошла успешно. Проверим теперь максимальные данные в таблице:

In [17]:
# выводим максимальные данные из таблицы data
data.max()

children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                          XNA
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

При выборе максимального значения есть странное значение столбца gender - XNA. Необходимо проверить сколько строк имеет странное значение данного столбца.  

In [18]:
# проверяем уникальные значение в столбце и их количество
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Такое значение только одно. Для дальнейшей работы с данными его можно удалить, так как вероятно это какие-то ошибочно внесенные данные. 

In [19]:
# удаляем из таблицы строку, в которой есть указанный пол, проверяем значения повторно
data = data.loc[data['gender'] !='XNA']
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [20]:
# повторно проверяем максимальное значение в таблице
data.max()

children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                            M
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

***Вывод***

Проверка на аномалии показала их наличие в трех столбцах - days_employed, gender и children. Большинство ошибок в данных вероятно связано с человеческим фактором. 
Отрицательные данные в столбце days_employed возможны в связи со спецификой расчета трудового стажа. 

Выявленные аномалии устранены.

### Шаг 2.3. Изменение типов данных.

Для удобства дальнейшей работы с данными необходимо преобразовать данные в столбцах days_employed и total_income в целочисленные.

In [21]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
display(data.head(10))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.4. Удаление дубликатов.

После обзора данных, работы с пропусками, аномалиями и приведения данных к общему виду необходимо проверить данные на наличие дубликатов. Исходя из первых 10 строк видно, что в столбце education присутствуют данные с неявными дубликатами. Необходимо привести их к общему значению перед началом обработки всей таблицы.



Для начала в столбце education сделаем все буквы строчными. Вероятно, при заполнении анкет данные указывались в той форме, в которой было удобно пользователю их внесшему и их не привели к единому формату.

In [22]:
# делаем все буквы строчными в столбце education и выводим таблицу data
data['education'] = data['education'].str.lower()
display(data.head(10))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Отлично. Все значения приведены к одному формату. Проверим дополнительно уникальные значения данного столбца, чтобы быть уверенным, что там нет ничего неожиданного. 

In [23]:
# проверяем уникальные значения столбца education
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Со значениями столбца education все в порядке.

In [24]:
# проверяем наличие дубликатов
data.duplicated().sum()

71

Действительно есть достаточное количество дубликатов. Их необходимо удалить, так как они будут мешать в процессе дальнейшей проверки гипотез:

In [25]:
# удаление явных дубликатов
data = data.drop_duplicates().reset_index(drop=True)

Проверяем, что дубликатов более нет:

In [26]:
# проверяем наличие дубликатов
data.duplicated().sum()

0

Дубликаты успешно удалены. 

***Вывод***

В прошлом пункте мы привели значения к целочисленным для удобства анализа. 
В рамках проверки на дубликаты мы:
- привели к единому регистру данные столбца education, проверили на возможное наличие не явных дубликатов
- после приведения данных к единым параметрам удалили явные дубликаты

Из причин появления дубликатов могу выделить прежде всего отсутствие конкретного формата заполнения данных - в итоге каждый человек при подаче анкеты на кредит заполняет данных по-своему. 
Явные дубли могли появиться в случае, если, например, один и тот же пользователь подал заявку несколько раз. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для удобства работы выделим данные по образованию и семейному положению в отдельные словари. Удалим значения из основной таблицы, чтобы она стала более простой и легкой по объему для анализа.

In [27]:
# создаем отдельную таблицу data_education со столбцами education_id и education
data_education = data[['education_id', 'education']]
# удаляем дубликаты из данной таблицы
data_education = data_education.drop_duplicates().reset_index(drop=True)
# выводим новую таблицу
display(data_education)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [28]:
# создаем отдельную таблицу data_family_status со столбцами family_status и family_status_id
data_family_status = data[['family_status_id', 'family_status']]
# удаляем дубликаты из данной таблицы
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)
# выводим новую таблицу 
display(data_family_status)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Таблицы с данными успешно созданны. Производим удаление столбцов family_status и education:

In [29]:
# сохраняем в таблице дата все столбцы кроме family_status и education
data = data[['children', 'days_employed', 'dob_years',
             'education_id', 'family_status_id','gender', 
             'income_type', 'debt', 'total_income', 'purpose']]

# выводим полученную таблицу на экран
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


***Вывод***

Для удобства дальнейшей работы сформированы два словаря. Уменьшена основная таблица. 

### Шаг 2.6. Категоризация дохода.

Для удобства анализа данных разделим всех граждан на категории в зависимости от уровня дохода:

    - 0–30000 — `'E'`;
    - 30001–50000 — `'D'`;
    - 50001–200000 — `'C'`;
    - 200001–1000000 — `'B'`;
    - 1000001 и выше — `'A'`.   

In [30]:
def total_group(total_income):
    
    '''
    Функция возвращает категорию граждан в зависимости от уровня дохода:
    Категория Е - пользователи с доходом меньше или равным 30 000
    Категория D - пользователи с доходом от 30 001 до 50 000 (включая эти значения)
    Категория C - пользователи с доходом от 50 001 до 200 000 (включая эти значения)
    Категория B - пользователи с доходом от 200 001 до 1 000 000 (включая эти значения)
    Категория А - пользователи с доходом больше или равным 1 000 001
    
    '''
    
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'

Добавим дополнительный столбец в таблицу data и внесем в него данные по категориям:

In [31]:
# добавляем в таблицу data столбец total_income_category 
# в него вносим значения из столбца total_income,к которым применим функцию
data['total_income_category'] = data['total_income'].apply(total_group)

# выведем таблицу с новым столбцом на экран
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


***Выводы***

Наиболее удобно для ответа на поставленные вопросы разделить граждан на категории в зависимости от уровня дохода. 
С помощью этого мы сможем в дальнейшем сгруппировать данные в зависимости от категории, и дать окончательные рекомендации на этом основании.

### Шаг 2.7. Категоризация целей кредита.

Еще на этапе удаления дубликатов мы отметили, что в столбце purpose есть неявные дубликаты. Вероятно, при заполнении анкет нет единой формы уточнения причин взятия кредита. Соответственно каждый заполняющий ее формулирует причину по-своему. 
Проверим какие есть уникальные значения в данных о цели взятия кредита:

In [32]:
# проверка уникальных значений столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все значения столбца с причинами взятия кредита можно объединить в три категории:

Образование: дополнительное образование, заняться образованием, получение образования, получение дополнительного образования, получение высшего образования, профильное образование, высшее образование, заняться высшим образованием

Недвижимость: покупка жилья, операции с жильем, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, строительство недвижимости, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, покупка своего жилья, операции с недвижимостью, покупка жилья для сдачи, ремонт жилью

Автомобиль: приобретение автомобиля, на покупку подержанного автомобиля, на покупку своего автомобиля, автомобили, сделка с подержанным автомобилем, свой автомобиль, сделка с автомобилем, на покупку автомобиля

Свадьба: сыграть свадьбу, на проведение свадьбы

Создадим функцию, чтобы привести разные формулировки причин взятия кредита к единому значению в зависимости от категории:

In [33]:
def purpose_categorizer(purpose):
    '''
    Функция возвращает новое более обобщенное название категории в зависимости от данных в столбце purpose:
    - если в столбце purpose есть слово 'автомобил' - 'операции с автомобилем'
    - если в столбце purpose есть слово 'свадьб' - 'проведение свадьбы'
    - если в столбце purpose есть слово 'образован' - 'получение образования'
    - если в столбце purpose нет ранее указанных значений - 'операции с недвижимостью'
    
    '''
    for row in data['purpose']:
            if 'автомобил' in purpose:
                return 'операции с автомобилем'
            if 'свадьб' in purpose:
                return 'проведение свадьбы'
            if 'образован' in purpose:
                return 'получение образования'
            return 'операции с недвижимостью'

Добавим в таблицу дополнительный столбец purpose_category и внесем в него обобщенную формулировку причины взятия кредита:

In [34]:
# создаем столбец purpose_category
# в него вносим значения столбца purpose, к которым применили функцию для категоризации причин взятия кредита
data['purpose_category'] = data['purpose'].apply(purpose_categorizer)

# выведем первые 5 строк таблицы, с новым столбцом
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Проверим, что все строки в новом столбце заполнены:

In [35]:
# выводим информацию о таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children                 21453 non-null int64
days_employed            21453 non-null int64
dob_years                21453 non-null int64
education_id             21453 non-null int64
family_status_id         21453 non-null int64
gender                   21453 non-null object
income_type              21453 non-null object
debt                     21453 non-null int64
total_income             21453 non-null int64
purpose                  21453 non-null object
total_income_category    21453 non-null object
purpose_category         21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


***Вывод***

Для дальнейшего корректного ответа на вопросы мы распределили всех граждан на 4 категории в зависимости от указанных ими причин взятия кредита. Это поможет нам при выводе данных. 

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на вопрос сформируем таблицу. Выведем данные о количестве заемщиков в зависимости от количества детей в их семье. Также разделим заемщиков на тех кто вернул кредит в срок (0) и на те кто не вернул (1). 

Добавим дополнительный столбец, чтобы увидеть долю тех, кто не вернул кредит вовремя, от общего числа заемщиков. 

Выведем данные на экран:

In [36]:
# создаем таблицу data_pivot_children
# в таблице отображаем количество заемщиков в зависимости от количества детей
data_pivot_children = data.pivot_table(index=['children'], columns='debt', values='education_id', aggfunc='count')
# добавляем столбец share_debtors, который показывает долю тех кто не вернул кредит в срок от общего числа заемщиков
data_pivot_children['share_debtors'] = data_pivot_children[1]/(data_pivot_children[1]+data_pivot_children[0])
# выводим таблицу
display(data_pivot_children)

debt,0,1,share_debtors
children,,,
0,13027.0,1063.0,0.075444
1,4410.0,445.0,0.091658
2,1858.0,194.0,0.094542
3,303.0,27.0,0.081818
4,37.0,4.0,0.097561
5,9.0,NaN,NaN
20,68.0,8.0,0.105263


Исходя из данных мы видим, что количество детей не слишком влияет на срок возврата кредита:
- В категориях граждан, у которых есть один или двое детей, доля не вернувших кредит в срок 9%
- В категориях граждан, у которых четверо или двадцать детей, доля невозврата - 10%
- В категории граждан с тремя детьми доля невозврата - 8%
- В категории граждан, у которых нет детей, доля тех, кто не вернул средства в срок - 7%

В категории с 5ю детьми данных по невозвратам кредита нет, поэтому посчитать долю нельзя. Можем лишь предположить, что доля невозвратов в их категории также будет на уровне 10%. 

##### Вывод 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сформируем таблицу с количеством заемщиков с разным семейным положением. Отдельно посчитаем тех, кто вернул средства в срок (0), и тех кто задержал выплаты (1). 
Добавим данные о доли заемщиков с разным семейным положением, не вернувших кредит вовремя, от общего числа заемщиков той же категории. 

Выведем данные:

In [37]:
# создаем таблицу data_pivot_family_status
# в таблице отображаем количество заемщиков в зависимости от id семейного статуса
data_pivot_family_status = data.pivot_table(index=['family_status_id'], columns='debt', values='education_id', aggfunc='count')
# добавляем столбец share_debtors с долей заемщиков, которые не вернули кредит в срок, от общего количества заемщиков в категории
data_pivot_family_status['share_debtors'] = data_pivot_family_status[1]/(data_pivot_family_status[1]+data_pivot_family_status[0])
# объединяем таблицу data_pivot_family_status с таблицей data_family_status
data_pivot_family_status = data_pivot_family_status.merge(data_family_status, on='family_status_id', how='left')
# для красоты визуализации удаляем столбец с family_status_id
data_pivot_family_status = data_pivot_family_status.drop('family_status_id',axis=1)
# выводим таблицу
display(data_pivot_family_status)

,0,1,share_debtors,family_status
0,11408,931,0.075452,женат / замужем
1,3762,388,0.093494,гражданский брак
2,896,63,0.065693,вдовец / вдова
3,1110,85,0.071130,в разводе
4,2536,274,0.097509,Не женат / не замужем


Большой разницы в возврате кредита в зависимости от семейного положения нет. 

Заемщики, которые ранее не были женаты\замужем, и заемщики, находящиеся в гражданском браке, немного чаще не выплачивают кредит в срок (9% не вернувших вовремя в каждой из категорий). 

Замужние\женатые заемщики, вдовцы\вдовы и те, кто в разводе возвращают кредит в срок совсем немного чаще (7% не вернувших вовремя в этих категориях)

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для проверки данной гипотезы создадим таблицу data_pivot_income. 
Выведем данные о количестве заемщиков в зависимости от дохода. Также разделим заемщиков на тех, кто вернул кредит в срок (0) и на те кто не вернул (1).

Добавим дополнительный столбец, чтобы увидеть долю тех, кто не вернул кредит вовремя от общего числа заемщиков.

Выведем данные:

In [38]:
# создаем таблицу data_pivot_income
# в таблице отображаем количество заемщиков в зависимости от категории дохода
data_pivot_income = data.pivot_table(index=['total_income_category'], columns='debt', values='education_id', aggfunc='count')
# добавляем столбец share_debtors с долей заемщиков, которые не вернули кредит в срок, от общего количества заемщиков в категории
data_pivot_income['share_debtors'] = data_pivot_income[1]/(data_pivot_income[1]+data_pivot_income[0])
# выведем данные на экран
display(data_pivot_income)

debt,0,1,share_debtors
total_income_category,,,
A,23,2,0.080000
B,4684,356,0.070635
C,14656,1360,0.084915
D,329,21,0.060000
E,20,2,0.090909


Сильной разницы в том, какая именно категория хуже возвращает кредит нет. Разрыв совсем небольшой.

Из текущих данных видно, что хуже всего возвращают кредиты заемщики с доходом менее или равным 30 000, и заемщики с доходами от 50 001 до 200 00 (9% не вернули в срок). 
Лучше остальных категорий возвращают средства в срок заемщики с доходом от 30 001 до 50 000 (6% не вернули в срок)

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Сформируем таблицу с количеством заемщиков с разными целями кредита. Отдельно посчитаем тех, кто вернул средства в срок (0), и тех кто задержал выплаты (1). Добавим данные о доли заемщиков с разными целями кредита, не вернувших кредит вовремя от общего числа заемщиков той же категории.

Выведем данные:

In [39]:
# создаем таблицу data_pivot_purpose
# в таблице отображаем количество заемщиков в зависимости от цели получения кредита
data_pivot_purpose = data.pivot_table(index=['purpose_category'], columns='debt', values='education_id', aggfunc='count')
# добавляем столбец share_debtors с долей заемщиков, которые не вернули кредит в срок, от общего количества заемщиков в категории
data_pivot_purpose['share_debtors'] = data_pivot_purpose[1]/(data_pivot_purpose[1]+data_pivot_purpose[0])
# выведем таблицу
display(data_pivot_purpose)

debt,0,1,share_debtors
purpose_category,,,
операции с автомобилем,3903,403,0.093590
операции с недвижимостью,10028,782,0.072340
получение образования,3643,370,0.092200
проведение свадьбы,2138,186,0.080034


В целом тут также нет большого разброса данных. Хуже возвращают кредиты те кто взял их на операции с автомобилем и на получение образования (9% не возвращают в срок).
Лучше остальных кредит в срок возвращают те, кто его брал на операции с недвижимостью (7% не возвращают в срок)

## Общий вывод:

Большая часть заемщиков возвращает кредиты в срок, поэтому разрывы между значениями крайне малы. 

***Наличие детей:***

Лучше всех возвращают кредиты в срок заемщики без детей (7,5% имеет просрочки по кредитам). 
Однако заемщики с тремя детьми также хорошо соблюдают сроки погашения (8% имеет просрочку по кредитам). 
Наибольший процент просрочек по кредитам (10%) имеют категории заемщиков, имеющие одного или двоих детей. 

***Семейное положение:***

Заемщики, которые ранее не были женаты\замужем, и заемщики, находящиеся в гражданском браке, немного чаще не выплачивают кредит в срок (9% не вернувших вовремя в каждой из категорий).
Замужние\женатые заемщики, вдовцы\вдовы и те, кто в разводе возвращают кредит в срок совсем немного чаще (7% не вернувших вовремя в этих категориях)

В целом те кто в браке или ранее был женат\замужем платят лучше, чем те кто не был. 

***Доход:***

Хуже всего возвращают средства в срок заемщики с заработком до 30 000 и те, кто получает от 50 001 до 200 000 (9% имеют просрочки по кредиту). Заемщики с доходом от 30 001 до 50 000 возвращают кредит в срок лучше остальных (6% не вернули в срок). 

***Цели кредита:***

Хуже возвращают кредиты те кто взял их на операции с автомобилем и на получение образования (9% не возвращают в срок). Лучше остальных кредит в срок возвращают те, кто его брал на операции с недвижимостью (7% не возвращают в срок)
